In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
from datetime import datetime
import time
import csv

class VnExpressCrawler:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.base_url = "https://vnexpress.net"
    
    def get_political_news_links(self, limit=5):
        """Lấy danh sách link bài báo chính trị mới nhất"""
        try:
            url = "https://vnexpress.net/thoi-su/chinh-tri"
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Tìm các link bài báo (cập nhật selector phù hợp với cấu trúc mới)
            article_links = []
            # Thử tìm các thẻ article có class 'item-news'
            articles = soup.find_all('article', class_='item-news')
            for article in articles:
                link_tag = article.find('a', href=True)
                if link_tag:
                    full_url = link_tag['href']
                    if not full_url.startswith('http'):
                        full_url = self.base_url + full_url
                    article_links.append(full_url)
                if len(article_links) >= limit:
                    break
            # Nếu không tìm thấy, fallback sang selector cũ
            if not article_links:
                articles = soup.find_all('h3', class_='title-news')
                for article in articles:
                    link_tag = article.find('a')
                    if link_tag and link_tag.get('href'):
                        full_url = link_tag['href']
                        if not full_url.startswith('http'):
                            full_url = self.base_url + full_url
                        article_links.append(full_url)
                    if len(article_links) >= limit:
                        break
            return article_links
            
        except Exception as e:
            print(f"Lỗi khi lấy danh sách bài báo: {e}")
            return []
    
    def extract_article_info(self, url):
        """Crawl thông tin chi tiết từ một bài báo"""
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Lấy tiêu đề
            title = ""
            title_tag = soup.find('h1', class_='title-detail')
            if title_tag:
                title = title_tag.get_text(strip=True)
            
            # Lấy tóm tắt
            summary = ""
            summary_tag = soup.find('p', class_='description')
            if summary_tag:
                summary = summary_tag.get_text(strip=True)
            
            # Lấy thời gian đăng
            publish_time = ""
            time_tag = soup.find('span', class_='date')
            if time_tag:
                publish_time = time_tag.get_text(strip=True)
            
            # Lấy nội dung chính
            content = ""
            content_div = soup.find('article', class_='fck_detail')
            if content_div:
                paragraphs = content_div.find_all('p', class_='Normal')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs[:3]])  # Lấy 3 đoạn đầu
            
            # Trích xuất các đối tượng được đề cập (tên người, chức vụ)
            mentioned_entities = self.extract_entities(title + " " + summary + " " + content)
            # Loại bỏ trường positions khỏi mentioned_entities
            if 'positions' in mentioned_entities:
                del mentioned_entities['positions']
            
            return {
                'url': url,
                'title': title,
                'summary': summary,
                'publish_time': publish_time,
                'content_preview': content,
                'mentioned_entities': mentioned_entities,
                'crawl_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            
        except Exception as e:
            print(f"Lỗi khi crawl bài báo {url}: {e}")
            return None
    
    def extract_entities(self, text):
        """Trích xuất các thực thể được đề cập trong bài báo"""
        entities = {
            'leaders': [],
            'organizations': [],
            'locations': [],
        }
        
        # Các pattern để tìm lãnh đạo và chức vụ
        leader_patterns = [
            r'Tổng [Bb]í thư ([A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][a-zàáảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữự]+ [A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][a-zàáảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữự]+)',
            r'Chủ tịch nước ([A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][a-zàáảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữự]+ [A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][a-zàáảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữự]+)',
            r'Thủ tướng ([A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][a-zàáảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữự]+ [A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][a-zàáảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữự]+)',
            r'Bộ trưởng ([A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][a-zàáảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữự]+ [A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎõọôốồổỗộơớờởỡợùúủũụưứừửữự][a-zàáảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữự]+)',
            r'Đại biểu ([A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][a-zàáảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữự]+ [A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][a-zàáảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữự]+)',
            r'Tổng thống ([A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][a-zA-Z\s]+)'
        ]
        
        for pattern in leader_patterns:
            matches = re.findall(pattern, text)
            entities['leaders'].extend(matches)
        
        # Tìm các tổ chức
        org_patterns = [
            r'(Quốc hội|Chính phủ|Hội đồng Quốc gia|Bộ Chính trị|Ban Chấp hành Trung ương|ASEAN|Đảng Cộng sản Việt Nam)',
            r'Bộ ([A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][a-zàáảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữự\s]+)'
        ]
        
        for pattern in org_patterns:
            matches = re.findall(pattern, text)
            if isinstance(matches[0] if matches else '', tuple):
                entities['organizations'].extend([match for match in matches])
            else:
                entities['organizations'].extend(matches)
        
        # Tìm địa danh
        location_patterns = [
            r'(Việt Nam|Hà Nội|TP HCM|Quảng Ngãi|Hungary|Pháp|Malaysia|Đông Nam Á)',
            r'(tỉnh|thành phố) ([A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][a-zàáảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữự\s]+)'
        ]
        
        for pattern in location_patterns:
            matches = re.findall(pattern, text)
            if matches:
                if isinstance(matches[0], tuple):
                    entities['locations'].extend([' '.join(match) for match in matches])
                else:
                    entities['locations'].extend(matches)
        
        # Loại bỏ duplicates
        for key in entities:
            entities[key] = list(set(entities[key]))
        
        return entities
    
    def crawl_news(self, limit=5):
        """Crawl tin tức chính trị từ VnExpress"""
        print(f"Bắt đầu crawl {limit} bài báo chính trị từ VnExpress...")
        
        # Lấy danh sách link
        article_links = self.get_political_news_links(limit)
        
        if not article_links:
            print("Không tìm thấy bài báo nào!")
            return []
        
        print(f"Tìm thấy {len(article_links)} bài báo. Bắt đầu crawl...")
        
        articles_data = []
        for i, link in enumerate(article_links, 1):
            print(f"Đang crawl bài {i}/{len(article_links)}: {link}")
            article_data = self.extract_article_info(link)
            
            if article_data:
                articles_data.append(article_data)
                print(f"✓ Crawl thành công: {article_data['title'][:50]}...")
            else:
                print(f"✗ Lỗi khi crawl bài báo")
            
            # Nghỉ 1 giây giữa các request để tránh bị block
            time.sleep(1)
        
        return articles_data
    
    
    def save_to_csv(self, data, filename="vnexpress_political_news.csv"):
        """Lưu dữ liệu vào file CSV"""
        try:
            with open(filename, 'w', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                # Header
                writer.writerow([
                    'url', 'title', 'summary', 'publish_time', 'content_preview', 'leaders', 'organizations', 'locations', 'crawl_time'
                ])
                for article in data:
                    writer.writerow([
                        article.get('url', ''),
                        article.get('title', ''),
                        article.get('summary', ''),
                        article.get('publish_time', ''),
                        article.get('content_preview', ''),
                        ', '.join(article.get('mentioned_entities', {}).get('leaders', [])),
                        ', '.join(article.get('mentioned_entities', {}).get('organizations', [])),
                        ', '.join(article.get('mentioned_entities', {}).get('locations', [])),
                        article.get('crawl_time', '')
                    ])
            print(f"Đã lưu dữ liệu vào file {filename}")
        except Exception as e:
            print(f"Lỗi khi lưu file CSV: {e}")
    
    def print_summary(self, articles_data):
        """In tóm tắt kết quả crawl"""
        print("\n" + "="*80)
        print("TÓM TẮT KẾT QUẢ CRAWL TIN TỨC CHÍNH TRỊ VNEXPRESS")
        print("="*80)
        
        for i, article in enumerate(articles_data, 1):
            print(f"\n🔸 BÀI {i}:")
            print(f"Tiêu đề: {article['title']}")
            print(f"URL: {article['url']}")
            print(f"Thời gian: {article['publish_time']}")
            print(f"Tóm tắt: {article['summary'][:100]}...")
            
            if article['mentioned_entities']['leaders']:
                print(f"Lãnh đạo được đề cập: {', '.join(article['mentioned_entities']['leaders'])}")
            
            if article['mentioned_entities']['organizations']:
                print(f"Tổ chức được đề cập: {', '.join(article['mentioned_entities']['organizations'])}")
            
            if article['mentioned_entities']['locations']:
                print(f"Địa danh được đề cập: {', '.join(article['mentioned_entities']['locations'])}")
            
            print("-" * 60)

# Sử dụng crawler
if __name__ == "__main__":
    crawler = VnExpressCrawler()
    
    # Crawl 5 bài báo chính trị mới nhất
    articles = crawler.crawl_news(limit=5)
    
    if articles:
        # In tóm tắt
        crawler.print_summary(articles)
        
        # Lưu vào file CSV
        crawler.save_to_csv(articles)
        
        print(f"\n✅ Hoàn thành crawl {len(articles)} bài báo!")
        print("📄 Dữ liệu đã được lưu vào file 'vnexpress_political_news.json' và 'vnexpress_political_news.csv'")
    else:
        print("❌ Không crawl được bài báo nào!")

Bắt đầu crawl 5 bài báo chính trị từ VnExpress...
Tìm thấy 5 bài báo. Bắt đầu crawl...
Đang crawl bài 1/5: https://vnexpress.net/ong-nguyen-van-tho-lam-pho-chu-tich-thuong-truc-ubnd-tp-hcm-4910346.html
✓ Crawl thành công: Ông Nguyễn Văn Thọ làm Phó chủ tịch thường trực UB...
Đang crawl bài 2/5: https://vnexpress.net/viet-nam-thai-lan-thuc-day-hop-tac-quoc-phong-thuc-chat-di-vao-chieu-sau-4910310.html
✓ Crawl thành công: Việt Nam - Thái Lan thúc đẩy hợp tác quốc phòng th...
Đang crawl bài 3/5: https://vnexpress.net/tong-bi-thu-yeu-cau-sua-chinh-sach-tien-luong-can-bo-cong-chuc-phu-hop-voi-mo-hinh-moi-4910138.html
✓ Crawl thành công: Tổng Bí thư yêu cầu sửa chính sách tiền lương cán ...
Đang crawl bài 4/5: https://vnexpress.net/cong-bo-6-bai-toan-lon-ve-khoa-hoc-cong-nghe-trong-linh-vuc-quoc-phong-4909909.html
✓ Crawl thành công: Công bố 6 bài toán lớn về khoa học công nghệ trong...
Đang crawl bài 5/5: https://vnexpress.net/noi-bien-voi-rung-quang-ngai-hoi-tu-nhieu-loi-the-4909395.html
✓